In [1]:
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from config import *
from typing import Tuple

from data.util import split_weekdays_and_weekends

from prediction_models.dbn import *
from prediction_models.kelm import *

%reload_ext autoreload
%autoreload 2

In [2]:
DBN_HIDDEN_LAYER_SIZES = CONFIG['DBN_HIDDEN_LAYER_SIZES']
GIBBS_SAMPLING_STEPS = CONFIG['GIBBS_SAMPLING_STEPS']
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)
TRAIN_START_DATE = datetime.strptime(CONFIG['TRAIN_START_DATE'], DATE_FORMAT)
TRAIN_END_DATE = datetime.strptime(CONFIG['TRAIN_END_DATE'], DATE_FORMAT)
TRAIN_TIME_WINDOW_SIZE = CONFIG['TRAIN_TIME_WINDOW_SIZE']

### Prepare training dataset

Load the matrix $C$ constructed from $Q$

In [3]:
mat_c = torch.load(out_path('mat_c.pt'))
TIME_DIM, SPACE_DIM = mat_c.shape
mat_c.shape

torch.Size([2880, 12])

Split $C$ into workdays and weekends data

In [4]:
mat_c_wd, mat_c_we = split_weekdays_and_weekends(mat_c, TRAIN_START_DATE, TRAIN_END_DATE)
assert mat_c_wd.shape[1] == mat_c_we.shape[1] == mat_c.shape[1]
assert mat_c_wd.shape[0] + mat_c_we.shape[0] == mat_c.shape[0]
mat_c_wd.shape, mat_c_we.shape

(torch.Size([2112, 12]), torch.Size([768, 12]))

Create a dataset for pre-training RBMs

In [5]:

class BinaryVectorDataset(Dataset):
    def __init__(self, n_bits: int):
        self.n_bits = n_bits
        self.format_str = f'{{0:0{self.n_bits}b}}'
    
    def __len__(self) -> int:
        return 2 ** self.n_bits
    
    def __getitem__(self, index: int) \
            -> Tuple[torch.TensorType, torch.TensorType]:
        bin_str = self.format_str.format(index)
        return torch.tensor([float(c) for c in bin_str]), torch.tensor([index,])

dbn_pre_train_loader = DataLoader(BinaryVectorDataset(SPACE_DIM), batch_size=256)

### Pre-train RBMs inside DBN

Train to reconstruct all possible combinations of 0's and 1's

In [6]:
dbn = DBN(SPACE_DIM, DBN_HIDDEN_LAYER_SIZES, k=GIBBS_SAMPLING_STEPS)
pre_train_dbm(dbn, dbn_pre_train_loader, print_every=1)

/home/kolusask/miniconda3/envs/.idp/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0, Machine 0:	Loss: 6.930359840393066
Epoch 0, Machine 1:	Loss: 8.915840148925781
Epoch 0, Machine 2:	Loss: 7.537132740020752
Epoch 1, Machine 0:	Loss: 2.977879047393799
Epoch 1, Machine 1:	Loss: 2.91357684135437
Epoch 1, Machine 2:	Loss: 3.262681007385254
Epoch 2, Machine 0:	Loss: 0.6644456386566162
Epoch 2, Machine 1:	Loss: -0.230025053024292
Epoch 2, Machine 2:	Loss: -0.4407786726951599
Epoch 3, Machine 0:	Loss: -0.5954934358596802
Epoch 3, Machine 1:	Loss: -0.9123860597610474
Epoch 3, Machine 2:	Loss: -1.5052132606506348
Epoch 4, Machine 0:	Loss: -1.2663729190826416
Epoch 4, Machine 1:	Loss: -1.0481911897659302
Epoch 4, Machine 2:	Loss: -1.585646390914917
Epoch 5, Machine 0:	Loss: -1.522771954536438
Epoch 5, Machine 1:	Loss: -1.1338024139404297
Epoch 5, Machine 2:	Loss: -1.5770678520202637
Epoch 6, Machine 0:	Loss: -1.621211051940918
Epoch 6, Machine 1:	Loss: -1.1400368213653564
Epoch 6, Machine 2:	Loss: -1.5527796745300293
Epoch 7, Machine 0:	Loss: -1.697648525238037
Epoch 7

DBN(
  (rbms): ModuleList(
    (0-2): 3 x RBM()
  )
)

In [23]:
# dbn(torch.tensor([11] * SPACE_DIM, dtype=torch.float32).unsqueeze(0))
dbn(torch.tensor([0, 1] * (SPACE_DIM // 2), dtype=torch.float32).unsqueeze(0))

tensor([[0.9523, 0.8238, 0.0757, 0.1101, 0.1491, 0.4945, 0.9101, 0.5890, 0.7142,
         0.3351, 0.5717, 0.3746]], grad_fn=<SigmoidBackward0>)

In [30]:
kelm = KELM()
print(mat_c.shape)
kelm.fit(mat_c, torch.tensor([11] * 2880, dtype=torch.float32))
display(mat_c)
kelm(mat_c)

torch.Size([2880, 12])


tensor([[1.3329e+01, 1.9828e-01, 1.3907e+01,  ..., 4.2834e+01, 4.2660e+00,
         3.7646e+00],
        [5.4583e+00, 2.0688e+00, 1.8704e+00,  ..., 2.2581e+01, 3.3491e+01,
         4.3524e+00],
        [1.9705e+00, 6.5596e+00, 4.6573e-02,  ..., 1.1451e+01, 1.1457e+02,
         1.9603e+01],
        ...,
        [6.8084e+01, 8.8494e+00, 1.0447e+01,  ..., 1.7520e+01, 7.4494e-01,
         9.1815e+00],
        [5.9604e+01, 5.7531e+00, 9.5811e+00,  ..., 4.4613e+00, 1.2693e+01,
         2.4498e+00],
        [1.5860e+02, 2.3826e-01, 5.6619e+00,  ..., 2.4567e+01, 2.2066e+01,
         3.1907e-02]])

tensor([[5.5000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 5.4987, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 5.5000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 5.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 5.5000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 5.5000]])

#### Train prediction of the first section

Define the loss function

In [8]:
loss_fn = nn.MSELoss()

In [58]:
kelm = KELM()
for start_time in range(TIME_DIM - TRAIN_TIME_WINDOW_SIZE - 1):
    end_time = start_time + TRAIN_TIME_WINDOW_SIZE
    train_window = mat_c[start_time:end_time] # [60, 12]
    label_window = mat_c[start_time + 1:end_time + 1][:, 0] # [60]
    features = dbn(train_window) # [60, 12]
    kelm.fit(features, label_window)
    predictions = kelm(features) # [60, 60]
    print(predictions.shape)
    print(loss_fn(predictions, label_window))
    # USE THIS TO TRAIN DBN

torch.Size([60, 60])
tensor(62.6212, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(62.5765, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(62.0669, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(64.9602, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(66.1223, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(65.6363, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(66.1174, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(68.9948, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(70.8755, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(68.6116, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(68.9507, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(68.8022, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(70.1512, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(70.0667, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tensor(70.6693, grad_fn=<MseLossBackward0>)
torch.Size([60, 60])
tens